https://www.plattform-i40.de/IP/Redaktion/EN/Downloads/Publikation/Details_of_the_Asset_Administration_Shell_Part1_V3.html

In [1]:
from rdflib import BNode, ConjunctiveGraph, Dataset, Graph, URIRef
from pyshacl import validate

# Construct AAS (template) from SHACL Shapes graph

In [2]:
prefixes = {
    'prov': 'http://www.w3.org/ns/prov#',
    'skos': 'http://www.w3.org/2004/02/skos/core#',
    'mas4ai': 'http://example.org/MAS4AI_GenericModel#',
    'aas': 'https://admin-shell.io/aas/3/0/RC01/',
    'aasenv': 'https://admin-shell.io/aas/3/0/RC01/AssetAdministrationShellEnvironment/',
    'aasaas': 'https://admin-shell.io/aas/3/0/RC01/AssetAdministrationShell/',
    'aassm': 'https://admin-shell.io/aas/3/0/RC01/Submodel/',
    'aassmc': 'https://admin-shell.io/aas/3/0/RC01/SubmodelElementCollection/',
    'aasrefer': 'https://admin-shell.io/aas/3/0/RC01/Referable/',
    'aasrel': 'https://admin-shell.io/aas/3/0/RC01/RelationshipElement/',
    'aasdata': 'https://admin-shell.io/aas/3/0/RC01/HasDataSpecification/',
    'aasprop': 'https://admin-shell.io/aas/3/0/RC01/Property/',
    'aasrange': 'https://admin-shell.io/aas/3/0/RC01/Range/',
    'aassem': 'https://admin-shell.io/aas/3/0/RC01/HasSemantics/',
    'aasref': 'https://admin-shell.io/aas/3/0/RC01/Reference/',
    'aaskey': 'https://admin-shell.io/aas/3/0/RC01/Key/',
    'aasida': 'https://admin-shell.io/aas/3/0/RC01/Identifiable/',
    'aaside': 'https://admin-shell.io/aas/3/0/RC01/Identifier/',
    'aaskeyt': 'https://admin-shell.io/aas/3/0/RC01/KeyType/',
    'aaskind': 'https://admin-shell.io/aas/3/0/RC01/HasKind/',
    'aasmod': 'https://admin-shell.io/aas/3/0/RC01/ModelingKind/'
}

def add_prefixes(graph):
    for k,v in prefixes.items():
        graph.namespace_manager.bind(k, URIRef(v))

## Initialize data model/graph

In [3]:
dataset = Dataset()

# file_name = 'Example_ServoDCMotor'
file_name = 'mas4aiDEMO'
g_sh = dataset.graph(identifier=URIRef('http://mas4ai.eu/id/graph/shapes'))
g_sh.parse(f'examples/{file_name}.shapes.ttl')

g_AAS_ont = dataset.graph(identifier=URIRef('https://admin-shell.io/aas/3/0/RC01/'))
g_AAS_ont.parse('https://raw.githubusercontent.com/admin-shell-io/aas-specs/master/schemas/rdf/rdf-ontology.ttl', format='text/turtle')
# g_AAS_ont.parse('https://raw.githubusercontent.com/admin-shell-io/aas-specs/draft-V3RC02-schemas/schemas/rdf/rdf-ontology.ttl', format='text/turtle')

g_AAS = dataset.graph(identifier=URIRef('http://mas4ai.eu/id/graph/aas'))

g_conj = ConjunctiveGraph(dataset.store)

### Enrich the shapes graph

#### Import referenced ontologies

In [4]:
for o in g_sh.objects(predicate=URIRef('http://www.w3.org/2002/07/owl#imports')):
    try:
        g_sh.parse(o.toPython())
    except:
        try:
            g_sh.parse('examples/' + o.toPython().split('/')[-1] + '.ttl')
        except FileNotFoundError:
            print('Cannot import ', o.toPython())

Cannot import  http://datashapes.org/dash


#### Add statements to indicate which classes should get an AAS

In [5]:
aas_classes = [
#     'http://example.org/ServoDCMotor',
    'http://www.tno.nl/mas4aiDEMO#RoboticArm',
]

for c in aas_classes:
    g_sh.add((URIRef(c), URIRef('http://example.org/MAS4AI_GenericModel#hasInterface'), BNode()))

#### Infer sh:group relation from rdfs:subPropertyOf

In [6]:
g_sh.update('''
INSERT {
  ?PropertyGroup a sh:PropertyGroup ;
    rdfs:label ?GroupLabel .
  ?PropertyShape sh:group ?PropertyGroup .
}
WHERE {
  ?PropertyShape a sh:PropertyShape ;
    sh:path/rdfs:subPropertyOf ?PropertyGroup .
}
''')

#### Infer properties from super classes

In [7]:
g_sh.update('''
INSERT {
  ?SubClass sh:property ?PropertyShape
  #?Property rdfs:domain ?SubClass
}
WHERE {
  VALUES ?PropertyType {owl:ObjectProperty owl:DatatypeProperty}
  ?Property a ?PropertyType ;
    rdfs:domain ?Class .
  ?PropertyShape sh:path ?Property.
  ?SubClass rdfs:subClassOf+ ?Class .
}
''')

## Construct AAS components
* Construct components, starting from the lowest level.
* Add (temporary) provenance data (`prov:wasDerivedFrom`) that can be used to link the different AAS components.

### Property
* The `sh:path` of a `sh:PropertyShape` should be mapped to the `aassem:semanticId` reference;
* A `sh:PropertyShape` without a `sh:class` is an `aas:Property`.

In [8]:
add_prefixes(dataset)

g_AAS.parse(data=g_sh.query('''
CONSTRUCT {
  # Property
  ?Property_iri a aas:Property ;
    aasprop:valueType ?dataType ;
    aassem:semanticId [
      a aas:Reference ;
      aasref:keys [
        a aas:Key ;
        aaskey:idType aaskeyt:IRI ;
        aaskey:value ?Property ;
      ] ;
    ] ;
    prov:wasDerivedFrom ?PropertyShape ;
  .
}
WHERE {
  ?PropertyShape a sh:PropertyShape ;
    sh:path ?Property ;
    sh:datatype ?dataType ;
  .

  FILTER NOT EXISTS { ?PropertyShape sh:class ?Class } #filter out reference properties

  BIND(iri(concat( "http://mas4ai.eu/id/property/template/", struuid() )) as ?Property_iri)
}
''').graph.serialize())

<Graph identifier=http://mas4ai.eu/id/graph/aas (<class 'rdflib.graph.Graph'>)>

### ReferenceElement
* A `sh:PropertyShape` that has a `sh:class`, is mapped to a `aas:ReferenceElement`.

In [9]:
add_prefixes(dataset)

g_AAS.parse(data=g_sh.query('''
BASE <http://mas4ai.eu/id/WP4/>

CONSTRUCT {
  # Reference Element
  ?ReferenceElement_iri a aas:ReferenceElement ;
    aasprop:valueType ?dataType ;
    aassem:semanticId [
      a aas:Reference ;
      aasref:keys [
        a aas:Key ;
        aaskey:idType aaskeyt:IRI ;
        aaskey:value ?ReferenceElement ;
      ] ;
    ] ;
    prov:wasDerivedFrom ?PropertyShape ;
  .
}
WHERE {
  ?PropertyShape a sh:PropertyShape ;
    sh:path ?ReferenceElement ;
    sh:class ?Class ; #filter on reference properties
  .

  FILTER EXISTS {?Class mas4ai:hasInterface []} #otherwise it should be a SMC

  BIND(iri(concat( "http://mas4ai.eu/id/referenceElement/template/", struuid() )) as ?ReferenceElement_iri)
}
''').graph.serialize())

<Graph identifier=http://mas4ai.eu/id/graph/aas (<class 'rdflib.graph.Graph'>)>

### Submodel Element Collection
* A `sh:PropertyShape` with `sh:maxCount > 1` should be embedded in an `aas:SubmodelElementCollection` (for example jobs) ;
* A `sh:NodeShape` of which the related `sh:targetClass` does not have `mas4ai:hasInterface` should be mapped to a `aas:SubmodelElementCollection` ('PropertyCollection').

#### 'Property Collection'

In [10]:
add_prefixes(dataset)

g_AAS.parse(data=g_sh.query('''
CONSTRUCT {
  # Submodel Element Collection (SMC)
  ?SMC_iri a aas:SubmodelElementCollection ;
    aassem:semanticId [
      a aas:Reference ;
      aasref:keys [
        a aas:Key ;
        aaskey:idType aaskeyt:IRI ;
        aaskey:value ?Class ;
      ] ;
    ] ;
    prov:wasDerivedFrom ?NodeShape ;
  .
}
WHERE {
  {
    ?NodeShape a sh:NodeShape ;
      sh:targetClass|rdfs:domain ?Class ;
    .
  } UNION {
    ?NodeShape a sh:NodeShape, rdfs:Class .
    BIND(?NodeShape as ?Class)
  } UNION {
    ?NodeShape a sh:NodeShape, owl:Class .
    BIND(?NodeShape as ?Class)
  }

  ?PropertyShape sh:class ?NodeShape .

  FILTER EXISTS {?NodeShape sh:property []}
  FILTER NOT EXISTS {?Class mas4ai:hasInterface []}
  FILTER NOT EXISTS {[] rdfs:subClassOf ?Class}

  BIND(iri(concat( "http://mas4ai.eu/id/smc/template/", struuid() )) as ?SMC_iri)
}
''').graph.serialize())

<Graph identifier=http://mas4ai.eu/id/graph/aas (<class 'rdflib.graph.Graph'>)>

#### Cardinality >1 properties

In [11]:
add_prefixes(dataset)

g_AAS.parse(data=g_sh.query('''
CONSTRUCT {
  # Submodel Element Collection (SMC)
  ?SMC_iri a aas:SubmodelElementCollection, mas4ai:MultiProperty ;
    aasrefer:idShort ?SMCidShort  ;
    aassem:semanticId [
      a aas:Reference ;
      aasref:keys [
        a aas:Key ;
        aaskey:idType aaskeyt:IRI ;
        aaskey:value ?Property ;
      ] ;
    ] ;
    prov:wasDerivedFrom ?PropertyShape ;
  .
}
WHERE {
  {
    ?PropertyShape a sh:PropertyShape ;
      sh:path?/rdfs:label ?propertyLabel ;
      sh:path ?Property ;
      sh:maxCount ?maxCount ;
    .
    FILTER( ?maxCount > 1 )
  } UNION {
    ?PropertyShape a sh:PropertyShape ;
      sh:path?/rdfs:label ?propertyLabel ;
      sh:path ?Property ;
    .
    FILTER NOT EXISTS { ?PropertyShape sh:maxCount [] } #if sh:maxCount>1 or not defined then embed the property in an ElementCollection
  }

  BIND(iri(concat( "http://mas4ai.eu/id/smc/template/", struuid() )) as ?SMC_iri)
  BIND(concat(?propertyLabel, 's') as ?SMC_idShort)
}
''').graph.serialize())

<Graph identifier=http://mas4ai.eu/id/graph/aas (<class 'rdflib.graph.Graph'>)>

##### Relation between SMCs and properties + reference elements + SMCs

In [12]:
add_prefixes(dataset)

g_conj.update('''
INSERT {
  GRAPH <http://mas4ai.eu/id/graph/aas> {
    ?SMC aassmc:value ?Value .
  }
}
WHERE {
  {
    ?SMC a aas:SubmodelElementCollection ;
      prov:wasDerivedFrom/sh:property ?PropertyShape .

    ?SubmodelElementType rdfs:subClassOf+ aas:SubmodelElement .
    ?Value a ?SubmodelElementType ;
      prov:wasDerivedFrom ?PropertyShape .
  } UNION {
    ?SMC a aas:SubmodelElementCollection, mas4ai:MultiProperty ;
      prov:wasDerivedFrom ?PropertyShape .
    ?PropertyShape a sh:PropertyShape ;
      sh:class?/^prov:wasDerivedFrom ?Value .
  }
  FILTER (?Value != ?SMC)
}
''')

### Submodel
* A `sh:PropertyGroup` is converted to an `aas:Submodel` and embeds all resources of type `sh:PropertyShape` that are linked to the `sh:PropertyGroup` via `sh:group`.

In [13]:
add_prefixes(dataset)

g_AAS.parse(data=g_sh.query('''
CONSTRUCT {
  # Submodel (SM)
  ?SM_iri a aas:Submodel ;
    prov:wasDerivedFrom ?PropertyGroup, ?PropertyShape ;
  .

#    aasida:identification [
#      a aas:Identifier ;
#      aaside:idType aaskeyt:IRI ;
#      aaside:identifier ?SM ;
#    ]

}
WHERE {
  {
    ?PropertyGroup a sh:PropertyGroup .
  } UNION {
    ?PropertyShape a sh:PropertyShape ;
      ^sh:property/sh:targetClass?/mas4ai:hasInterface [] ; # only properties that are directly related to a class that requires an AAS
    .
    FILTER NOT EXISTS {?PropertyShape sh:group []}
  }

  BIND(iri(concat( "http://mas4ai.eu/id/sm/template/", struuid() )) as ?SM_iri)
}
''').graph.serialize())

<Graph identifier=http://mas4ai.eu/id/graph/aas (<class 'rdflib.graph.Graph'>)>

##### Relation between submodels and properties + submodel element collections

In [14]:
add_prefixes(dataset)

g_conj.update('''
INSERT {
  GRAPH <http://mas4ai.eu/id/graph/aas> {
    ?Submodel aassm:submodelElements ?SubmodelElement .
  }
}
WHERE {
  {
    ?Submodel a aas:Submodel ;
      prov:wasDerivedFrom/^sh:group? ?PropertyShape .

    VALUES ?ElementType {aas:Property aas:SubmodelElementCollection}
    ?SubmodelElement a ?ElementType ;
      prov:wasDerivedFrom ?PropertyShape .
  } UNION {
    ?Submodel a aas:Submodel ;
      prov:wasDerivedFrom/^sh:group? ?PropertyShape .

    ?PropertyShape a sh:PropertyShape .
    {
      ?PropertyShape sh:class/^sh:targetClass ?NodeShape . #via SHACL
    } UNION {
      ?PropertyShape sh:path/(rdfs:domain|rdfs:range) ?NodeShape . #via ontology
    }

    ?SubmodelElement a aas:SubmodelElementCollection ;
      prov:wasDerivedFrom ?NodeShape .

    FILTER NOT EXISTS {?PropertyShape sh:group []}
  }

  #exclude nested submodel element collections
  FILTER NOT EXISTS {
    [] aassmc:value ?SubmodelElement .
    ?SubmodelElement a aas:SubmodelElementCollection .
  }
}
''')

### AssetAdministrationShell

In [15]:
add_prefixes(dataset)

g_AAS.parse(data=g_sh.query('''
CONSTRUCT {
  # Asset Administration Shell (AAS)
  ?AAS_iri a aas:AssetAdministrationShell ;
    aassem:semanticId [
      a aas:Reference ;
      aasref:keys [
        a aas:Key ;
        aaskey:idType aaskeyt:IRI ;
        aaskey:value ?AASClass ;
      ] ;
    ] ;
    prov:wasDerivedFrom ?NodeShape ;
  .
}
WHERE {
  {
    ?NodeShape a sh:NodeShape, rdfs:Class .
    BIND(?NodeShape as ?AASClass)
  } UNION {
    ?NodeShape a sh:NodeShape, owl:Class .
    BIND(?NodeShape as ?AASClass)
  } UNION {
    ?NodeShape a sh:NodeShape ;
      sh:targetClass ?AASClass ;
    .
  }

  ?AASClass mas4ai:hasInterface [] .

  BIND(iri(concat( "http://mas4ai.eu/id/aas/template/", struuid() )) as ?AAS_iri)
}
''').graph.serialize())

<Graph identifier=http://mas4ai.eu/id/graph/aas (<class 'rdflib.graph.Graph'>)>

##### Relations between asset administration shell and submodels

In [16]:
add_prefixes(dataset)

g_conj.update('''
INSERT {
  GRAPH <http://mas4ai.eu/id/graph/aas> {
    ?AAS aasaas:submodels ?Submodel .
  }
}
WHERE {
  ?AAS a aas:AssetAdministrationShell ;
    prov:wasDerivedFrom ?NodeShape .

  ?NodeShape a sh:NodeShape ;
    rdfs:subClassOf*/sh:property/sh:group? ?ShapeGroup .

  ?Submodel a aas:Submodel ;
    prov:wasDerivedFrom ?ShapeGroup .
}
''')

### AssetAdministrationShellEnvironment

In [17]:
add_prefixes(dataset)

g_AAS.update('''
INSERT {
  <http://mas4ai.eu/def/WP4/AASEnv> a aas:AssetAdministrationShellEnvironment ;
    aasenv:assetAdministrationShells ?AAS ;
    aasenv:submodels ?Submodel .
}
WHERE {
  ?AAS a aas:AssetAdministrationShell ;
    aasaas:submodel ?Submodel .

  ?Submodel a aas:Submodel .
}
''')

### Add common statements for objects of a certain type
(if they don't exist yet)

#### HasKind
`aaskind:kind`

In [18]:
g_conj.update('''
INSERT {
  GRAPH <http://mas4ai.eu/id/graph/aas> {
    ?Object aaskind:kind aasmod:Template .
  }
}
WHERE {
  ?Object a/rdfs:subClassOf* aas:HasKind .
  FILTER NOT EXISTS { ?Object aaskind:kind [] }
}
''')

#### Referable
|Shapes predicate|AAS predicate|
|---|---|
| `rdfs:label` | `aasrefer:idShort`     |
| `rdfs:comment` | `aasrefer:description` |
| `skos:prefLabel` | `aasrefer:displayName` |

In [19]:
g_conj.update('''
INSERT {
  GRAPH <http://mas4ai.eu/id/graph/aas> {
    ?Object aasrefer:idShort ?idShort ;
      rdfs:label ?shapeLabel ;
      aasrefer:description ?description ;
      aasrefer:displayName ?displayName ;
    .
  }
}
WHERE {
  ?Object a/rdfs:subClassOf* aas:Referable ;
    prov:wasDerivedFrom ?Shape .

  ?Shape sh:path?/rdfs:label ?shapeLabel .
  OPTIONAL { ?Object aasrefer:idShort ?_idShort }
  BIND ( REPLACE(COALESCE(?_idShort, ?shapeLabel), " ", "_") AS ?idShort )

  OPTIONAL {
    ?Shape sh:path?/rdfs:comment ?shapeComment .
    OPTIONAL { ?Object aasrefer:description ?_description }
    BIND ( COALESCE(?_description, ?shapeComment) AS ?description )
  }

  OPTIONAL {
    ?Shape sh:path?/skos:prefLabel ?shapePrefLabel .
    OPTIONAL { ?Object aasrefer:displayName ?_displayName }
    BIND ( COALESCE(?_displayName, ?shapePrefLabel) AS ?displayName )
  }
}
''')

## Inspect AAS graph

In [20]:
print(g_AAS.serialize())

@prefix aas: <https://admin-shell.io/aas/3/0/RC01/> .
@prefix aasaas: <https://admin-shell.io/aas/3/0/RC01/AssetAdministrationShell/> .
@prefix aaskey: <https://admin-shell.io/aas/3/0/RC01/Key/> .
@prefix aaskeyt: <https://admin-shell.io/aas/3/0/RC01/KeyType/> .
@prefix aaskind: <https://admin-shell.io/aas/3/0/RC01/HasKind/> .
@prefix aasmod: <https://admin-shell.io/aas/3/0/RC01/ModelingKind/> .
@prefix aasprop: <https://admin-shell.io/aas/3/0/RC01/Property/> .
@prefix aasref: <https://admin-shell.io/aas/3/0/RC01/Reference/> .
@prefix aasrefer: <https://admin-shell.io/aas/3/0/RC01/Referable/> .
@prefix aassem: <https://admin-shell.io/aas/3/0/RC01/HasSemantics/> .
@prefix aassm: <https://admin-shell.io/aas/3/0/RC01/Submodel/> .
@prefix aassmc: <https://admin-shell.io/aas/3/0/RC01/SubmodelElementCollection/> .
@prefix mas4ai: <http://example.org/MAS4AI_GenericModel#> .
@prefix mas4aiDEMO: <http://www.tno.nl/mas4aiDEMO#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http:/

In [21]:
# Remove provenance statements
g_AAS.update('''
DELETE {
  ?s prov:wasDerivedFrom ?o
}
WHERE {
  ?s prov:wasDerivedFrom ?o
}
''')

g_AAS.update('''
DELETE {
  ?s a mas4ai:MultiProperty
}
WHERE {
  ?s a mas4ai:MultiProperty
}
''')

# Store to file
g_AAS.serialize(f'examples/{file_name}_template.aas.ttl')
# g_AAS.serialize('examples/f'examples/{file_name}_template.aas.jsonld', format='json-ld')

<Graph identifier=http://mas4ai.eu/id/graph/aas (<class 'rdflib.graph.Graph'>)>